In [ ]:

%pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128

In [ ]:
%pip install opencv-python-headless
%pip install opencv-python
%pip install scikit-image
%pip install scikit-learn
%pip install tqdm
%pip install sympy==1.13.3

In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def compute_mean_std(data_dir, image_size=(256, 256), batch_size=64):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize(image_size),
        transforms.ToTensor()
    ])

    dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    mean = 0.
    std = 0.
    nb_samples = 0.

    for data, _ in loader:
        batch_samples = data.size(0)
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        nb_samples += batch_samples

    mean /= nb_samples
    std /= nb_samples

    return mean, std


train_paths = {
    "Axial ADNI": "D:/Licenta/Datasets/ADNI_Oficial/Processed/Axial/Train",
    "Sagittal ADNI": "D:/Licenta/Datasets/ADNI_Oficial/Filtered/Sagittal/Train",
    "Parkinson PPMI": "D:/Licenta/Datasets/PPMI_Oficial/Augmented/Train"
}

for name, path in train_paths.items():
    print(f"\nProcesare: {name}")
    mean, std = compute_mean_std(path)
    print(f"Mean : {mean.tolist()}")
    print(f"Std  : {std.tolist()}")


In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import os

alz_classes = ["AD", "CN", "EMCI", "LMCI", "MCI"]
park_classes = ["Control", "PD", "Prodromal", "SWEDD"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform_axial = transforms.Compose([
   transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.2006, 0.2006, 0.2006], [0.2396, 0.2396, 0.2396])
])

transform_sagittal = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.2487, 0.2487, 0.2487], [0.2599, 0.2599, 0.2599])
])

transform_parkinson = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.2514, 0.2514, 0.2514], [0.2475, 0.2475, 0.2475])
])



In [ ]:
import torch.nn as nn
import torchvision.models as models

class ResNetModel(nn.Module):
    def __init__(self, pretrained=True, num_classes=5):
        super(ResNetModel, self).__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1 if pretrained else None)

        for name, param in self.resnet.named_parameters():
            if "layer3" in name or "layer4" in name or "fc" in name:
                param.requires_grad = True
            else:
                param.requires_grad = False

        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.BatchNorm1d(1024),
            nn.SiLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.SiLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)


In [ ]:
import torch.nn as nn
import torchvision.models as models

class ResNet101_MRI(nn.Module):
    def __init__(self, pretrained=True, num_classes=5):
        super(ResNet101_MRI, self).__init__()
        self.resnet = models.resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V1 if pretrained else None)

        for name, param in self.resnet.named_parameters():
            if "layer2" in name or "layer3" in name or "layer4" in name or "fc" in name:
                param.requires_grad = True
            else:
                param.requires_grad = False

        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)


In [ ]:

alz_ax_50 = ResNetModel(pretrained=False, num_classes=5)
alz_ax_50.load_state_dict(torch.load("ResNet_Alzheimer_Axial_Multiclass.pth"))
alz_ax_50.to(device).eval()

alz_ax_101 = ResNet101_MRI(pretrained=False, num_classes=5)
alz_ax_101.load_state_dict(torch.load("ResNet101_Alzheimer_Axial_Multiclass.pth"))
alz_ax_101.to(device).eval()

alz_sag_50 = ResNetModel(pretrained=False, num_classes=5)
alz_sag_50.load_state_dict(torch.load("ResNet_Alzheimer_Sagittal_Multiclass.pth"))
alz_sag_50.to(device).eval()

alz_sag_101 = ResNet101_MRI(pretrained=False, num_classes=5)
alz_sag_101.load_state_dict(torch.load("ResNet101_Alzheimer_Sagittal_Multiclass.pth"))
alz_sag_101.to(device).eval()

park_50 = ResNetModel(pretrained=False, num_classes=4)
park_50.load_state_dict(torch.load("ResNet_Parkinson_Multiclass.pth"))
park_50.to(device).eval()

park_101 = ResNet101_MRI(pretrained=False, num_classes=4)
park_101.load_state_dict(torch.load("ResNet101_Parkinson_Multiclass.pth"))
park_101.to(device).eval()


In [ ]:
def predict_with_ensemble_fixed(model_r50, model_r101, img_tensor, class_names, w_r50=0.5, w_r101=0.5):
   
    model_r50.eval()
    model_r101.eval()

    with torch.no_grad(), torch.amp.autocast(device_type=img_tensor.device.type, enabled=(img_tensor.device.type == 'cuda')):
        out_r50 = model_r50(img_tensor)
        out_r101 = model_r101(img_tensor)
        combined_logits = w_r50 * out_r50 + w_r101 * out_r101
        probs = torch.softmax(combined_logits, dim=1)[0]
        pred_idx = probs.argmax().item()
        score = probs[pred_idx].item()
        pred_label = class_names[pred_idx]
    
    return pred_idx, pred_label, score


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
import torch

def evaluate_ensemble_fixed(model_r50, model_r101, dataloader, device, class_names, w_r50=0.5, w_r101=0.5, save_report=False, report_path="ensemble_report.txt"):
    assert abs(w_r50 + w_r101 - 1.0) < 1e-5, "Weights must sum to 1.0"

    model_r50.eval()
    model_r101.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            with torch.amp.autocast(device_type=device.type, enabled=(device.type == 'cuda')):
                out_r50 = model_r50(images)
                out_r101 = model_r101(images)
                combined_logits = w_r50 * out_r50 + w_r101 * out_r101
                _, predicted = torch.max(combined_logits, dim=1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)

    print(f"\nEnsemble (ResNet50 + ResNet101) Accuracy: {acc * 100:.2f}%")
    print("\nClassification Report:\n")
    print(report)

    if save_report:
        with open(report_path, "w") as f:
            f.write(f"Ensemble Accuracy: {acc * 100:.2f}%\n\n")
            f.write(report)

    return acc, all_preds, all_labels


In [ ]:
axial_loader = DataLoader(
    datasets.ImageFolder("D:/Licenta/Datasets/ADNI_Oficial/Processed/Axial/Test", transform=transform_axial),
    batch_size=32, shuffle=False
)


sagittal_loader = DataLoader(
    datasets.ImageFolder("D:/Licenta/Datasets/ADNI_Oficial/Filtered/Sagittal/Test", transform=transform_sagittal),
    batch_size=32, shuffle=False
)

parkinson_loader = DataLoader(
    datasets.ImageFolder("D:/Licenta/Datasets/PPMI_Oficial/Augmented/Test", transform=transform_parkinson),
    batch_size=32, shuffle=False
)

In [ ]:
print("\n--- Axial ---")
evaluate_ensemble_fixed(
    model_r50=alz_ax_50,
    model_r101=alz_ax_101,
    dataloader=axial_loader,
    device=device,
    class_names=alz_classes,
    w_r50=0.5,
    w_r101=0.5,
    save_report=True,
    report_path="ensemble_axial_report.txt"
)

print("\n--- Sagittal ---")
evaluate_ensemble_fixed(
    model_r50=alz_sag_50,
    model_r101=alz_sag_101,
    dataloader=sagittal_loader,
    device=device,
    class_names=alz_classes,
    w_r50=0.5,
    w_r101=0.5,
    save_report=True,
    report_path="ensemble_sagittal_report.txt"
)


print("\n--- Parkinson ---")
evaluate_ensemble_fixed(
    model_r50=park_50,
    model_r101=park_101,
    dataloader=parkinson_loader,
    device=device,
    class_names=park_classes,
    w_r50=0.5,
    w_r101=0.5,
    save_report=True,
    report_path="ensemble_parkinson_report.txt"
)


In [ ]:
import os
import random
import math
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

def count_test_images(axial_dir, sagittal_dir, park_dir):
    def count_images(folder):
        count = 0
        for cls in os.listdir(folder):
            cls_path = os.path.join(folder, cls)
            if os.path.isdir(cls_path):
                count += sum(fname.endswith(".png") for fname in os.listdir(cls_path))
        return count

    axial_count = count_images(axial_dir)
    sagittal_count = count_images(sagittal_dir)
    park_count = count_images(park_dir)
    total = axial_count + sagittal_count + park_count

    print(f"Axial: {axial_count} imagini")
    print(f"Sagittal: {sagittal_count} imagini")
    print(f"Parkinson: {park_count} imagini")
    print(f"Total imagini în setul de test: {total}")
    return total

def predict_from_folders_ensemble_visual(axial_dir, sagittal_dir, park_dir, total=100, cols=6, device='cuda'):
    device = torch.device(device)

    def collect_images_with_labels(folder, tag, classes):
        images = []
        for cls in os.listdir(folder):
            cls_path = os.path.join(folder, cls)
            if not os.path.isdir(cls_path): continue
            for f in os.listdir(cls_path):
                if f.endswith('.png'):
                    images.append({
                        "path": os.path.join(cls_path, f),
                        "label": cls,
                        "tag": tag,
                        "classes": classes
                    })
        return images

    axial_images = collect_images_with_labels(axial_dir, 'axial', alz_classes)
    sagittal_images = collect_images_with_labels(sagittal_dir, 'sagittal', alz_classes)
    park_images = collect_images_with_labels(park_dir, 'parkinson', park_classes)

    print("\n--- Evaluare Axial ---")
    axial_loader = DataLoader(ImageFolder(axial_dir, transform=transform_axial), batch_size=32, shuffle=False)
    evaluate_ensemble_fixed(alz_ax_50, alz_ax_101, axial_loader, device, alz_classes, 0.5, 0.5, True, "report_axial.txt")

    print("\n--- Evaluare Sagittal ---")
    sagittal_loader = DataLoader(ImageFolder(sagittal_dir, transform=transform_sagittal), batch_size=32, shuffle=False)
    evaluate_ensemble_fixed(alz_sag_50, alz_sag_101, sagittal_loader, device, alz_classes, 0.5, 0.5, True, "report_sagittal.txt")

    print("\n--- Evaluare Parkinson ---")
    parkinson_loader = DataLoader(ImageFolder(park_dir, transform=transform_parkinson), batch_size=32, shuffle=False)
    evaluate_ensemble_fixed(park_50, park_101, parkinson_loader, device, park_classes, 0.5, 0.5, True, "report_parkinson.txt")

    all_images = axial_images + sagittal_images + park_images
    random.shuffle(all_images)
    selected = all_images[:total]

    rows = math.ceil(total / cols)
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 2.5, rows * 2.5))
    axs = axs.flatten()
    incorrect = []

    for i, img_info in enumerate(selected):
        img_path = img_info["path"]
        label = img_info["label"]
        tag = img_info["tag"]
        classes = img_info["classes"]

        img = Image.open(img_path).convert("L")

        if tag == 'axial':
            transform = transform_axial
            model_r50, model_r101 = alz_ax_50, alz_ax_101
        elif tag == 'sagittal':
            transform = transform_sagittal
            model_r50, model_r101 = alz_sag_50, alz_sag_101
        else:
            transform = transform_parkinson
            model_r50, model_r101 = park_50, park_101

        img_tensor = transform(img).unsqueeze(0).to(device)
        pred_idx, pred_label, score = predict_with_ensemble_fixed(model_r50, model_r101, img_tensor, classes)

        is_correct = (pred_label == label)
        color = "green" if is_correct else "red"
        title = f"{tag} | Pred: {pred_label}\nReal: {label}"

        axs[i].imshow(img, cmap='gray')
        axs[i].axis('off')
        axs[i].set_title(title, fontsize=7, color=color)

        if not is_correct:
            incorrect.append((img_path, label, pred_label, score))

    for j in range(len(selected), len(axs)):
        axs[j].axis("off")

    plt.tight_layout()
    plt.suptitle("Predicții Corecte (Verde) / Greșite (Roșu)", fontsize=16)
    plt.subplots_adjust(top=0.93)
    plt.show()

    print(f"\nTotal greșite în subsetul afișat: {len(incorrect)} din {total}")
    accuracy = 1 - len(incorrect) / total
    print(f"Acuratețea pe subsetul afișat: {accuracy * 100:.2f}%")

    if incorrect:
        print("\nExemple greșite:")
        for path, real, pred, score in incorrect:
            print(f"{os.path.basename(path)} | Real: {real} | Pred: {pred} | Score: {score:.2f}")


In [ ]:
count_test_images(
    "D:/Licenta/Datasets/ADNI_Oficial/Processed/Axial/Test",
    "D:/Licenta/Datasets/ADNI_Oficial/Filtered/Sagittal/Test",
    "D:/Licenta/Datasets/PPMI_Oficial/Augmented/Test"
)

In [ ]:
predict_from_folders_ensemble_visual(
    axial_dir="D:/Licenta/Datasets/ADNI_Oficial/Processed/Axial/Test",
    sagittal_dir="D:/Licenta/Datasets/ADNI_Oficial/Filtered/Sagittal/Test",
    park_dir="D:/Licenta/Datasets/PPMI_Oficial/Augmented/Test",
    total=6028,
    cols=5,
    device='cuda'
)
